In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import ipywidgets as widgets
from ipywidgets import interact

# Parameters
lx    = 5                       
nu    = 0.1                    
tau   = 1.0 / (3 * nu + 0.5)    

cs    = 1/np.sqrt(3)            
rho1  = 0.5
rho2  = 1

#opposite discrete velocities for HBB
c_opp = np.array([0,1,3])

f    = np.zeros((Q,lx))
rho  = np.array([rho1 for i in range(0,int(lx/2))] + [rho2 for i in range(int(lx/2),lx)])


### A.PonD

One dimensional discrete velocities $c_k$ for the $D1Q3$ model

In [1]:
#Discrete velocities ck
c = np.array([0,1,-1])
D = 1
Q = 3

The model is characterized by the lattice temperature $T_L$ and the weights $\omega_k$ associated with $c_k$

In [ ]:
# lattice weights
w = np.array([2./3.,1./6.,1./6.])

with $\omega_k$ are he weights of the Gauss-Hermite quadrature.

The particles velocities $v_k$ are defined relative to a reference frame with the discrete velocities $c_k$, specified by the frame velocity $u$ and the reference temperature $T$

$$v_k = \sqrt{\dfrac{T }{T_L}} c_k+ u$$

The co-moving referance frame is specified by the local temperature $T_{ref} = T(x,t)$ and the local flow velocity $u_{ref} = u(x,t)$

In [ ]:
f    = np.zeros((Q,lx))
u    = np.zeros(lx)
T    = np.zeros((Q,lx))

vk = np.sqrt(T/TL)*c + u

### B. Kinetic equations

Single relaxation time BGK, two-population kinectic model for ideal gas ¿¿with variable adiabatic exponent ??

$$\partial_t f_k + v_k \cdot \nabla f_k = \dfrac{1}{\tau} (f_k^{eq} - f_k) $$

$$\partial_t f_k + v_k \cdot \nabla g_k = \dfrac{1}{\tau} (g_k^{eq} - g_k) $$

where $f_k^{eq}$ and $g_k^{eq}$ are the local equilibrium distribution functions and $\tau$ is the relaxation time.

In the co-moving reference frame, the equilibrium populations depend only on the density and temperature (Kalikunis et.al 2022)

$$f_k^{eq} = \rho \omega_i$$

$$g_k^{eq} = \left(C_v- \dfrac{D}{2}\right) T \rho \omega_i$$

In [ ]:
#add return here        
def collision(f,rho,u,tau):
    feq = f_eq(rho,u)
    Teq = T_eq(rho,u)
    f += (1/tau) * (feq - f) 
    T += (1/taut)* (Teq - T)
    
    
#streaming

In [ ]:
#equilibrium distribution function pseudopotential LBM
def f_eq(c,rho,u):
    f_eq = np.zeros((Q,lx))
    for k in range(Q):
        f_eq[k] = rho*w[k]*(1 + 3*(c[k]*u) + 4.5*((c[k]*u)**2) - 1.5*(u**2))
    return f_eq

def T_eq(rho,u):
    T_eq = np.zeros((Q,lx))
    for k in range(Q):
        T_eq[k] = T*w[k]*(1+3*(c[k]*u))
        
#vectorized edf
feq = w[:,np.newaxis]*rho*(1 + 3*(c[:,np.newaxis]*u) + 4.5*((c[:,np.newaxis]*u)**2) - 1.5*(u**2)) 
feq

Local conservation laws for the density $\rho$, momentum $\rho u$ and the total energy $\rho E$ are

$$\rho = \sum_{k=0}^{Q-1} f_k = \sum_{k=0}^{Q-1} f_k^{eq} $$

$$\rho u = \sum_{k=0}^{Q-1} v_k f_k = \sum_{k=0}^{Q-1} v_ k f_k^{eq} $$

$$\rho E = \sum_{k=0}^{Q-1} \dfrac{{v_k}^2}{2} f_k + \sum_{k=0}^{Q-1} g_k = \sum_{k=0}^{Q-1}\dfrac{{v_k}^2}{2}f_k^{eq} + \sum_{k=0}^{Q-1} g_k^{eq}  $$


where the total energy of ideal gas is

$$\rho E = C_v \rho T + \dfrac{\rho u^2}{2}$$

with $C_v$ the specific  heat at constant volume.

In [ ]:
rho   = np.sum(f,axis=0)
u     = np.dot(c,f)/rho  
rhoE  = np.sum(T,axis=0) + np.sum((vk/2)*f,axis=0)

### Reference frame transformation

Transformation of population $f_{k}^\lambda$ with respect to a $\lambda$-reference to a different reference frame $\lambda '$ with the discrete velocities $v_k^{\lambda'}$

$$v_k^{\lambda'} = \sqrt{\dfrac{T_{ref}' }{T_L}} c_k+ u_{ref}'$$

In [ ]:
vk = np.sqrt(T/TL)*c + u
vk_p = np.sqrt(T_p/TL)*c + u_p

The transformation is a linear operation, based on the moments matching

$$f_k^{\lambda'} = \mathcal{M}_{\lambda'}^{-1} \mathcal{M}_{\lambda}f^\lambda = \mathcal{G}_\lambda^{\lambda'} f^\lambda $$

In [ ]:
Transfer_Matrix = np.linalg.inv(vk_p)*vk

Populations are reconstructed at a point $x$ and time $t$ using Lagrange interpolation

$$f_k^{\lambda} = \sum_{s=1}^{p} a_s (x-x_s) \mathcal{G}_{\lambda_s}^{\lambda'} f^{\lambda_s}(x_s,t) $$

$$g_k^{\lambda} = \sum_{s=1}^{p} a_s (x-x_s) \mathcal{G}_{\lambda_s}^{\lambda'} g^{\lambda_s}(x_s,t) $$